### Pre-Process

In [1]:
%reset -f
import pandas as pd
from collections import deque
import numpy as np

c1 = [7,7,7,3,3,8,2,6,6,4,4,1,1]
c2 = [3,6,4,8,2,0,0,4,1,1,5,5,2]
 
data_array = np.vstack((c1,c2)).T
del c1 ,c2


Adj_List = [[] for i in  range(data_array.max()+1)]#無向圖  
Adj_DList = [[] for i in  range(data_array.max()+1)]#有向圖
Adj_RDList = [[] for i in  range(data_array.max()+1)]#反向圖

#Prepare Graph data
for i,j in data_array:
    #無向圖
    Adj_List[i].append(j)
    Adj_List[j].append(i)
    #有向圖
    Adj_DList[i].append(j)
    #反向圖
    Adj_RDList[j].append(i)

In [4]:
Adj_List

[[8, 2],
 [6, 4, 5, 2],
 [3, 0, 1],
 [7, 8, 2],
 [7, 6, 1, 5],
 [4, 1],
 [7, 4, 1],
 [3, 6, 4],
 [3, 0]]

In [5]:
Adj_DList

[[], [5, 2], [0], [8, 2], [1, 5], [], [4, 1], [3, 6, 4], [0]]

In [6]:
Adj_RDList

[[8, 2], [6, 4], [3, 1], [7], [7, 6], [4, 1], [7], [], [3]]

### Initial

In [2]:
visit=np.zeros(data_array.max()+1,'i4') #遍歷in的時間
visit_point=np.zeros(data_array.max()+1,'i4') #紀錄遍歷順序的array

#儲存區/ Init:自己
parent =np.zeros(data_array.max()+1,'i4') #紀錄父節點:遍歷過程中上一層的點
p = np.zeros(data_array.max()+1,'i4') #parent的tmp
sdom = np.zeros(data_array.max()+1,'i4') #(tmp)入邊中能回朔到的最高點(最高的意思就是最早遍歷)->所以有可能是入邊(或是其他點的best)
best = np.zeros(data_array.max()+1,'i4') #idom的tmp/用來存當前運算時的支配點
idom = np.zeros(data_array.max()+1,'i4') #運算時:找i不如找idom[i](次低支配點)/最後儲存支配點的array

idom_T = np.zeros(data_array.max()+1,'i4')

for i in range(data_array.max()+1):
    parent[i] = p[i] = best[i] = sdom[i] = idom[i] = i;
revsdom=[[] for i in range(data_array.max()+1)] 
#組長紀錄成員的list,組長是成員入邊中所能找到的最高點

### DFS

In [3]:
t=0
#Depth-First Search Main Function 
def DFS_list(x,Adj):
    global t
    #bottom line:
    if (visit[x]!=0):return;
    
    #start
    visit_point[t]=x
    t+=1      
    visit[x] = t
    for j in Adj[x]:
        if (visit[j]==0):
            DFS_list(j,Adj)
            parent[j] = x


DFS_list(7,Adj_DList)

In [7]:
visit_point

array([7, 3, 8, 0, 2, 6, 4, 1, 5])

In [8]:
visit

array([4, 8, 5, 2, 7, 9, 6, 1, 3])

### Dominator

In [9]:
def find(x):
    if (x==p[x]):
        return x
    y = find(p[x])
    #如果父節點best[p[x]]有入邊比best[x]早,
    #這樣以後找best[x]就不如找best[p[x]]好了(DP)
    if (visit[sdom[best[p[x]]]] < visit[sdom[best[x]]]):
        print('find更新:','把best[p[x]]:',best[p[x]],'->存到best[x]:',best[x])
        best[x] = best[p[x]]
    p[x] = y
    return p[x]

def DFS_list_Dominator(root,visit_member,Adj_list,Adj_Rlist):
    #尋找支配點,某點j的支配點定義為,若拔掉該支配點,則從root開始必走不到j這點
    #應用層面:拔掉支配點,家戶會破碎->支配點同時影響兩個家庭
    #Step 1:
    for k in range(t-1,0,-1):#根節點沒做
        i = visit_member[k];
        print('------------')
        print('New Stage:',i)
        #先看最早入邊有沒有比i早的,更新i的sdom
        for j in Adj_Rlist[i]:#check j指向i的有沒有更早的入邊(sdom)
            print('j:',j)
            if (visit[j]):
                find(j)
                #如果best[j]的入邊時間比i的最早入邊還早,就更新i的最早入邊呀
                if (visit[sdom[best[j]]] < visit[sdom[i]]):
                    sdom[i] = sdom[best[j]]#更新i的組長是誰
                    print('i:',i,' 更新最早入邊:',sdom[best[j]])
        revsdom[sdom[i]].append(i)#組長記錄組員;
        #在每一次i的迴圈中,會透過動態的調整best並即時確認best
        print('i入邊最早的成員有:','Lead(sdom[i]):',sdom[i],
              '/Members(revsdom[sdom[i]]):',revsdom[sdom[i]],'New In:',i)
        #把當前i當子樹頭，更新組員j們中idom
        #會有兩種情境:
        #第一種(if):如果j的best比i高,則先暫訂該j為i的idom(支配點)
        #第二種(else):如果組員j的best沒比組長i高,則該組員的idom(支配點)必是組長i
        for j in revsdom[i]:
            find(j) #透過find函數去更新best,p
            #如果組員j的best比我小,則j的支配點(idom[j])是best[j];反之就是我i
            if visit[sdom[best[j]]] < visit[i]:
                idom[j]=best[j]
            else:
                idom[j]=i
        #更新父節點
        for j in Adj_list[i]: #紀錄j的父節點p 
            if (parent[j] == i):
                p[j] = i;
    #Step 2:
    #只補做根節點=root這一個點,根節點可能是某個開頭,並不是全域最早
    for j in revsdom[root]:
        find(j)
        if visit[sdom[best[j]]] < visit[root]:
            idom[j]=best[j]
        else:
            idom[j]=root
    #Step 3:
    # 時刻從小到大，依序完成idom[best[j]]
    print('---------------------------------------','\n'
          'Step 3:時刻從小到大，依序完成idom的遞推更新')
    for k in range(t):
        i = visit_member[k];
        print('i:',i,'idom[i]:',idom[i],'sdom[i]:',sdom[i])
        if idom[i] != sdom[i]: 
            #當i的idom(次低支配點)不等於i的sdom(入邊能回朔到的最高點)
            idom[i] = idom[idom[i]] #把頭的idom一步一步往下傳給各個成員
    #Step 4:
    #歸戶需要,支配點中,出邊必須兩條以上才被認為是支配點    
    for i in list(set(idom)):
        if len(Adj_List[idom[i]])>1:
            idom_T[i]=1

In [10]:
DFS_list_Dominator(root=7,
                   visit_member=visit_point,
                   Adj_list=Adj_List,
                   Adj_Rlist=Adj_RDList)

------------
New Stage: 5
j: 4
i: 5  更新最早入邊: 4
j: 1
i入邊最早的成員有: Lead(sdom[i]): 4 /Members(revsdom[sdom[i]]): [5] New In: 5
------------
New Stage: 1
j: 6
i: 1  更新最早入邊: 6
j: 4
i入邊最早的成員有: Lead(sdom[i]): 6 /Members(revsdom[sdom[i]]): [1] New In: 1
------------
New Stage: 4
j: 7
i: 4  更新最早入邊: 7
j: 6
i入邊最早的成員有: Lead(sdom[i]): 7 /Members(revsdom[sdom[i]]): [4] New In: 4
find更新: 把best[p[x]]: 1 ->存到best[x]: 5
------------
New Stage: 6
j: 7
i: 6  更新最早入邊: 7
i入邊最早的成員有: Lead(sdom[i]): 7 /Members(revsdom[sdom[i]]): [4, 6] New In: 6
find更新: 把best[p[x]]: 4 ->存到best[x]: 1
------------
New Stage: 2
j: 3
i: 2  更新最早入邊: 3
j: 1
i: 2  更新最早入邊: 7
i入邊最早的成員有: Lead(sdom[i]): 7 /Members(revsdom[sdom[i]]): [4, 6, 2] New In: 2
------------
New Stage: 0
j: 8
i: 0  更新最早入邊: 8
j: 2
i: 0  更新最早入邊: 7
i入邊最早的成員有: Lead(sdom[i]): 7 /Members(revsdom[sdom[i]]): [4, 6, 2, 0] New In: 0
------------
New Stage: 8
j: 3
i: 8  更新最早入邊: 3
i入邊最早的成員有: Lead(sdom[i]): 3 /Members(revsdom[sdom[i]]): [8] New In: 8
------------
New Stage: 3
j: 7

In [11]:
idom

array([7, 7, 7, 7, 7, 7, 7, 7, 3])